In [2]:
import subprocess # cmd 명령어를 파이썬코드에서 실행시켜줌
from pydub import AudioSegment # pydub을 이용하여 긴 audio파일을 여러개의 짧은 audio파일로 쪼갤 수 있음
import math
import openai
import glob # 디렉토리 안에서 특정한 이름을 갖는 파일을 찾을 수 있음

def extract_audio_from_video(video_path, audio_path):
    # -i : 입력, -vn : 비디오 제거 (video nope)
    command = ["ffmpeg","-i",video_path,"-vn",audio_path]    
    subprocess.run(command)

def cut_audio_in_chunks(audio_path, chunk_size, chunks_folder):
    track = AudioSegment.from_mp3(audio_path)
    chunk_len = chunk_size * 60 * 1000 # ms단위 # chunk_size는 분 단위 시간을 입력
    chunks = math.ceil(len(track)/chunk_len)

    # audio파일을 특정 시간길이(chunk_len)로 잘라냄
    for i in range(chunks):
        start_time = i*chunk_len
        end_time = (i+1)*chunk_len
        chunk = track[start_time:end_time]
        chunk.export(f"{chunks_folder}/chunk_{i}.mp3",format="mp3")

def transcribe_chunks(chunk_folder, destination):
    files = glob.glob(f"{chunk_folder}/*.mp3")
    for file in files:
        # # openai.Audio.strascribe의 whisper 모델을 이용하여 오디오에서 텍스트를 추출할 수 있음
        # 파일 사용 뒤에 closed 되도록 with문을 이용해서 파일 오픈하겠음
        # rb : read as binary 모드, a: append 모드 (문자열을 하나의 파일에 계속 이어붙이기 위해 append모드로 열었음)
        with open(file,"rb") as audio_file, open(destination,"a") as text_file:
            transcript = openai.Audio.transcribe("whisper-1", audio_file)
            text_file.write(transcript["text"])

In [ ]:
extract_audio_from_video("./files/podcast.mp4","./files/podcast.mp3")

In [3]:
cut_audio_in_chunks("./files/podcast.mp3", 10, "./files/chunks")

In [8]:
transcribe_chunks("./files/chunks","./files/transcript.txt")